In [3]:
import datetime
import numpy as np
import pandas as pd
from pandas import Series
from itertools import groupby
import matplotlib.pyplot as plt

%matplotlib inline  

In [4]:
df = pd.read_csv('../../data/test.csv')
ids = df['KPI ID'].unique()

In [5]:
df.head()

,timestamp,value,KPI ID
0,1501475700,2.719298,02e99bd4f6cfb33f
1,1501475760,2.456140,02e99bd4f6cfb33f
2,1501475820,2.333333,02e99bd4f6cfb33f
3,1501475880,2.543860,02e99bd4f6cfb33f
4,1501475940,2.333333,02e99bd4f6cfb33f


In [7]:
df['label'].value_counts()

In [8]:
df['value'].describe()

count    2.345211e+06
mean     9.744256e+07
std      3.019166e+08
min     -6.003177e+00
25%      4.850000e+00
50%      1.411000e+03
75%      2.498000e+03
max      1.632326e+09
Name: value, dtype: float64

In [9]:
# upsampling
dfs = []
for _id in ids:
    df_tmp = df[df['KPI ID'] == _id]
    df_tmp['datetime'] = pd.to_datetime(df_tmp['timestamp'], unit='s')
    df_tmp = df_tmp.set_index('datetime')
    df_tmp = df_tmp.sort_index()
    df_tmp = df_tmp.resample('1T').ffill()
    
    df_tmp['id'] = df_tmp['KPI ID'] + '-' + df_tmp.index.astype('str') 
    df_tmp = df_tmp.set_index('id')
    
    dfs.append(df_tmp)
    print(f'Done with {_id}')
    
df_sampled = pd.concat(dfs)

/Users/alex/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Done with 02e99bd4f6cfb33f


/Users/alex/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Done with 9bd90500bfd11edb
Done with da403e4e3f87c9e0
Done with a5bf5d65261d859a
Done with 18fbb1d5a5dc099d
Done with 09513ae3e75778a3
Done with c58bfcbacb2822d1
Done with 1c35dbf57f55f5e4
Done with 046ec29ddf80d62e
Done with 07927a9a18fa19ae
Done with 54e8a140f6237526
Done with b3b2e6d1a791d63a
Done with 8a20c229e9860d0c
Done with 769894baefea4e9e
Done with 76f4550c43334374
Done with e0770391decc44ce
Done with 8c892e5525f3e491
Done with 40e25005ff8992bd
Done with cff6d3c01e6a6bfa
Done with 71595dd7171f4540
Done with 7c189dd36f048a6c
Done with a40b1df87e3f1c87
Done with 8bef9af9a922e0b3
Done with affb01ca2b4f0b45
Done with 9ee5879409dccef9
Done with 88cf3a776ba00e7c


In [11]:
df_sampled.head()

,timestamp,value,KPI ID
id,,,
02e99bd4f6cfb33f-2017-07-31 04:35:00,1501475700,2.719298,02e99bd4f6cfb33f
02e99bd4f6cfb33f-2017-07-31 04:36:00,1501475760,2.456140,02e99bd4f6cfb33f
02e99bd4f6cfb33f-2017-07-31 04:37:00,1501475820,2.333333,02e99bd4f6cfb33f
02e99bd4f6cfb33f-2017-07-31 04:38:00,1501475880,2.543860,02e99bd4f6cfb33f
02e99bd4f6cfb33f-2017-07-31 04:39:00,1501475940,2.333333,02e99bd4f6cfb33f


In [13]:
df_sampled[df_sampled['KPI ID'] == ids[1]].describe()

,timestamp,value
count,1.317950e+05,131795.000000
mean,1.505418e+09,24.456852
std,2.284245e+06,7.386116
min,1.501476e+09,0.000000
25%,1.503453e+09,19.250000
50%,1.505430e+09,24.416666
75%,1.507406e+09,29.666666
max,1.509383e+09,103.166664


In [15]:
# normalization
for _id in ids:
    df_tmp = df_sampled[df_sampled['KPI ID'] == _id]
    df_diff = df_tmp['value'].diff()
    #df_tmp = df_tmp[['timestamp', 'label', 'KPI ID']].join(df_diff)
    df_tmp = df_tmp[['timestamp', 'KPI ID']].join(df_diff)
    df_sampled.loc[df_sampled['KPI ID'] == _id, 'value'] = (df_tmp['value'] - df_tmp['value'].mean()) / df_tmp['value'].std()

In [17]:
df_sampled[df_sampled['KPI ID'] == ids[9]].describe()

,timestamp,value
count,6.327100e+04,6.327000e+04
mean,1.472876e+09,-7.743029e-16
std,1.084169e+06,1.000000e+00
min,1.470991e+09,-1.325987e+02
25%,1.471875e+09,4.285743e-04
50%,1.472889e+09,4.285743e-04
75%,1.473838e+09,4.285743e-04
max,1.474787e+09,8.827266e+00


In [19]:
df_sampled.loc[df_sampled['value'].isnull(), 'value'] = 0.0

In [21]:
df_sampled.to_csv('../../data/test_snd.csv')